In [ ]:
import torch.utils.data as data
import os
import PIL.Image as Image
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import albumentations as A
import torch
from albumentations.pytorch import ToTensorV2
import joblib
test_image_path = '../input/happywhale-resized-dataset/test_images_512x512'
train_path = "../input/happywhale-resized-dataset/train_images_512x512"
transform = {

    "train": A.Compose([

        # pil.image to tensor
        #A.HorizontalFlip(p=0.5),
        #A.VerticalFlip(p=0.5),
        A.Resize(384,384),
        #A.OneOf([A.MotionBlur(p=0.2),
                 #A.MedianBlur(p=0.2),
                 #A.Blur(blur_limit=3, p=0.2)], p=0.5),
        #A.GaussNoise(),
        #A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.5),
        #A.RandomBrightnessContrast(p=0.5),
        A.Normalize(mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225], ),
        ToTensorV2(),  # torchvision.transforms.Normalize(mean, std, inplace=False)
    ]),
    # tensor([0.4560, 0.4245, 0.3905], device='cuda:0') tensor([0.2755, 0.2671, 0.2602], device='cuda:0')
    "valid": A.Compose([

        A.Resize(384,384),
        A.Normalize(mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225], ),
        ToTensorV2(),  # torchvision.transforms.Normalize(mean, std, inplace=False)
    ])
}


class WhaleDataset(data.Dataset):

    def __init__(self,train_csv, mode, transform=None):

        if mode == 'train':


            # print(train_csv.head())
            train_img = train_csv.image
            # print(train_img)

            labels = train_csv.individual_key
            imgs = []
            for file, label in zip(train_img, labels):
                imgs.append([file, label])

            self.imgs = imgs

            # self.transform = transform
            self.mode = mode
        if mode == 'valid':
            pass
        if mode == 'test':
            train_img = train_csv.image
            imgs = []
            for file in train_img:
                imgs.append(file)

            self.imgs = imgs

            # self.transform = transform
            self.mode = mode

    def __getitem__(self, idx: int):

        # x是jpg格式，label是png
        if self.mode == "train":
            x_path, label = self.imgs[idx]
            pilimage = Image.open(os.path.join(train_path, x_path)).convert("RGB")
            pilimage = np.array(pilimage)
            pilimage = transform["train"](image=pilimage)
            return pilimage["image"], label
        
        if self.mode == "valid":
            x_path, label = self.imgs[idx]
            pilimage = Image.open(os.path.join(opt.train_path, x_path)).convert("RGB")
            pilimage = np.array(pilimage)
            pilimage = transform["valid"](image=pilimage)
            return pilimage["image"], label
        if self.mode == "test":
            x_path = self.imgs[idx]
            pilimage = Image.open(os.path.join(test_image_path, x_path)).convert("RGB")
            pilimage = np.array(pilimage)
            pilimage = transform["valid"](image=pilimage)
            return pilimage["image"]

        # if self.transform is not None:
        # img_x = self.transform(img_x)
        # plt.imshow(pilimage)
        # plt.show()

    def __len__(self):
        return len(self.imgs)


In [ ]:
import sys
sys.path.append("../input/timm-20220211/pytorch-image-models-master")
import timm
#sys.path.append('../input/convnext/ConvNeXt')\
!pip install faiss-gpu
import faiss

In [ ]:
import torch
import torch.nn as nn
from torchvision.models.resnet import resnet50
import timm
import time
import math
from scipy import spatial
from tqdm import tqdm
import warnings
import cv2
import pandas as pd
import numpy as np
from numpy import dot, sqrt
import seaborn as sns
import matplotlib as mpl
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from IPython.display import display_html

from sklearn.model_selection import StratifiedKFold
import torch
from torch.utils.data import DataLoader, Dataset
from torch.optim import Adam, lr_scheduler
import torch.nn as nn
import torch.nn.functional as F

num_classes = 15587
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def l2_norm(input, axis=1):
    norm = torch.norm(input, 2, axis, True)
    output = torch.div(input, norm)
    return output


def softmax_loss(results, labels):
    labels = labels.view(-1)
    loss = F.cross_entropy(results, labels, reduce=True)
    return loss

class MagrginLinear(nn.Module):
    # implementation of additive margin softmax loss in https://arxiv.org/abs/1801.05599
    def __init__(self, embedding_size=512, classnum=10008,  s=64., m=0.5):
        super(MagrginLinear, self).__init__()
        self.classnum = classnum
        self.kernel = nn.Parameter(torch.Tensor(embedding_size,classnum))
        # initial kernel
        self.kernel.data.uniform_(-1, 1).renorm_(2,1,1e-5).mul_(1e5)
        self.m = m # the margin value, default is 0.5
        self.s = s # scalar value default is 64, see normface https://arxiv.org/abs/1704.06369
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.mm = self.sin_m * m  # issue 1
        self.threshold = math.cos(math.pi - m)


    def forward(self, embbedings, label, is_infer = False):
        # weights norm
        nB = len(embbedings)
        kernel_norm = l2_norm(self.kernel,axis=0)
        # cos(theta+m)
        cos_theta = torch.mm(embbedings, kernel_norm)
#         output = torch.mm(embbedings,kernel_norm)
        cos_theta = cos_theta.clamp(-1,1) # for numerical stability
        cos_theta_2 = torch.pow(cos_theta, 2)
        sin_theta_2 = 1 - cos_theta_2
        sin_theta = torch.sqrt(sin_theta_2)
        cos_theta_m = (cos_theta * self.cos_m - sin_theta * self.sin_m)

        # this condition controls the theta+m should in range [0, pi]
        #      0<=theta+m<=pi
        #     -m<=theta<=pi-m

        cond_v = cos_theta - self.threshold
        cond_mask = cond_v <= 0
        keep_val = (cos_theta - self.mm) # when theta not in [0,pi], use cosface instead
        cos_theta_m[cond_mask] = keep_val[cond_mask]
        output = cos_theta * 1.0 # a little bit hacky way to prevent in_place operation on cos_theta
        idx_ = torch.arange(0, nB, dtype=torch.long)

        if not is_infer:
            output[idx_, label] = cos_theta_m[idx_, label]

        output *= self.s # scale up in order to make softmax work, first introduced in normface
        return output


class ArcMarginProduct(nn.Module):
    r"""Implement of large margin arc distance: :
        Args:
            in_features: size of each input sample
            out_features: size of each output sample
            s: norm of input feature
            m: margin
            cos(theta + m)
        """

    def __init__(self, in_features, out_features, s=30.0,
                 m=0.50, easy_margin=False, ls_eps=0.0):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.ls_eps = ls_eps  # label smoothing
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))

        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------
        # one_hot = torch.zeros(cosine.size(), requires_grad=True, device='cuda')
        one_hot = torch.zeros(cosine.size(), device=device)
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.out_features
        # -------------torch.where(out_i = {x_i if condition_i else y_i) ------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s

        return output
class BinaryHead(nn.Module):

    def __init__(self, num_class=15587, emb_size = 2048, s = 16.0):
        super(BinaryHead,self).__init__()
        self.s = s
        self.fc = nn.Sequential(nn.Linear(emb_size, num_class))

    def forward(self, fea):
        fea = l2_norm(fea)
        logit = self.fc(fea)*self.s
        return logit

class MarginHead(nn.Module):

    def __init__(self, num_class=15587, emb_size = 2048, s=64., m=0.5):
        super(MarginHead,self).__init__()
        self.fc = MagrginLinear(embedding_size=emb_size, classnum=num_class , s=s, m=m)

    def forward(self, fea, label, is_infer):
        fea = l2_norm(fea)
        logit = self.fc(fea, label, is_infer)
        return logit


class HappyWhaleModel(nn.Module):
    def __init__(self, modelName, numClasses, noNeurons, embeddingSize):
        super(HappyWhaleModel, self).__init__()
        # @self.fea_extra_layer = [2, 3]
        self.model = timm.create_model(modelName, pretrained=False, features_only=True,
                                       # out_indices=self.fea_extra_layer
                                       )
        self.embsize = embeddingSize
        # in_features = self.model.classifier.in_features
        in_features = 1920
        # self.model.classifier = nn.Identity()
        # self.model.global_pool = nn.Identity()
        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.poolingl2 = nn.AdaptiveAvgPool2d(1)
        self.poolingl3 = nn.AdaptiveAvgPool2d(1)
        self.poolingl4 = nn.AdaptiveAvgPool2d(1)
        self.poolingl5 = nn.AdaptiveAvgPool2d(1)

        self.fc = nn.Sequential(
            nn.BatchNorm1d(in_features),
            nn.Dropout(p=0.2, inplace=False),
            nn.Linear(in_features, embeddingSize),
            nn.BatchNorm1d(embeddingSize),
        )

        # self.avgpool = nn.AvgPool1d(kernel_size=3, stride=2, padding=1)
        self.arc_head = ArcMarginProduct(in_features=embeddingSize, out_features=numClasses, m=0.3)
        #self.class_head = BinaryHead(numClasses, emb_size=embeddingSize, s=16.0)

    def forward(self, images, labels=None):
        features = self.model(images)

        # pooled_features = self.pooling(features).flatten(1)
        # pooled_drop = self.drop(pooled_features)
        features[0] = self.pooling(features[0])
        features[1] = self.poolingl2(features[1])
        features[2] = self.poolingl3(features[2])
        features[3] = self.poolingl4(features[3])
        #features[4] = self.poolingl5(features[4])
        emb = torch.cat(features, dim=1)
        emb = emb.flatten(1)
        emb = self.fc(emb)

        if labels != None:

            arc_output = self.arc_head(emb, labels)
            #class_output = self.class_head(emb)
            return arc_output, emb
        else:

            return emb  # feartures

In [ ]:
if __name__ == "__main__":
    import os
    import gc
    import pandas as pd
    import torch
    from torchvision.transforms import transforms

    from torch.utils.data import DataLoader
    from tqdm import tqdm

    import matplotlib.pyplot as plt
    import numpy as np
    import time
    from sklearn.model_selection import StratifiedKFold, KFold
    from sklearn.model_selection import train_test_split
    from sklearn.neighbors import NearestNeighbors
    from sklearn.preprocessing import normalize
    import sklearn.metrics.pairwise as smp


    import torch.nn as nn
    import torch.nn.functional as F
    from torch.autograd import Variable
    import joblib

    N_SPLITS = 5
    MODEL_NAME = 'convnext_base_384_in22ft1k'
    NUM_CLASSES = 15587
    NO_NEURONS = 250
    EMBEDDING_SIZE = 512

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    torch.cuda.manual_seed_all(seed=3407)
    # cross_validation
    df_train = pd.read_csv("../input/train-encoded/train_encoded.csv")
    df_test = pd.read_csv("../input/happy-whale-and-dolphin/sample_submission.csv")
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=6)
    for fold_id, (train_idx, val_idx) in enumerate(kfold.split(df_train["image"], df_train["individual_key"])):
            if fold_id!=0:
                break
            model = HappyWhaleModel(MODEL_NAME, NUM_CLASSES, NO_NEURONS, EMBEDDING_SIZE).to(device)
            train_data = WhaleDataset(df_train, "train")
            train_loader = DataLoader(train_data, batch_size=64, num_workers=2, shuffle=False)
            test_data = WhaleDataset(df_test, "test")
            test_loader = DataLoader(test_data, batch_size=64, num_workers=2, shuffle=False)

            model.load_state_dict(torch.load("../input/c-base-day322/model_13_0.720611.pth")["net"])

            model.eval()
            k = 100
            thres = 0.7
            train_embeddings = []
            trn_lbl_list = []

            threshold = thres

            with torch.no_grad():
                for data in tqdm(train_loader):
                    image, target = data
                    image, target = image.to(device), target.to(device)
                    embedding = model(image, labels = None)

                    train_embeddings.extend(embedding.detach().cpu().numpy())
                    trn_lbl_list.append(target.detach().cpu().numpy().tolist())

            image_embeddings = np.array(train_embeddings)
            trn_lbl_list = np.concatenate(trn_lbl_list)
            test_embeddings = []
            with torch.no_grad():
                for data in tqdm(test_loader):
                    image = data
                    image = image.to(device)
                    embedding = model(image, labels=None)

                    test_embeddings.extend(embedding.detach().cpu().numpy())

            test_image_embeddings = np.array(test_embeddings)
            
            
            train_emb = normalize(image_embeddings, axis=1, norm='l2')
            test_emb = normalize(test_image_embeddings, axis=1, norm='l2')
            index = faiss.IndexFlatIP(EMBEDDING_SIZE)
            train_embeds = train_emb
            test_embeds = test_emb
            index.add(train_embeds)
            dist, idx = index.search(test_embeds, k=100)
            
            confs = 1 - dist
            # === PREDICTION ===

            sum_top5 = 0.0
            pbar = tqdm(range(len(test_image_embeddings)))
            map_ = 0.0
            predictions = []
            preds_decoded = {}
            for i in pbar:
                index = idx[i][:5]
                conf = confs[i][:5]
                # preds = np.array(trn_lbl_list)[index]
                preds = df_train.iloc[index]["individual_id"].values
                #if conf[0] < threshold:
                #    templist = ['new_individual', preds[1], preds[2], preds[3], preds[4]]
                #elif conf[1] < threshold:
                #    templist = [preds[0], 'new_individual', preds[2], preds[3], preds[4]]
                #elif conf[2] < threshold:
                #    templist = [preds[0], preds[1], 'new_individual', preds[3], preds[4]]
                #elif conf[3] < threshold:
                #    templist = [preds[0], preds[1], preds[2], 'new_individual', preds[4]]
                if conf[4] < threshold:
                    templist = [preds[0], preds[1], preds[2], preds[3], 'new_individual']
                else:
                    templist = preds

                preds_decoded[df_test.iloc[i]["image"]] = templist

            for x in tqdm(preds_decoded):
                preds_decoded[x] = ' '.join(preds_decoded[x])



            predictions = pd.Series(preds_decoded).reset_index()
            predictions.columns = ['image', 'predictions']
            predictions.to_csv('submission.csv', index=False)
            predictions.head()
